<a href="https://colab.research.google.com/github/Narachii/ml_test/blob/test_branch_for_colab/test_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread
from gspread.models import Cell
import pandas as pd
import numpy as np
import sys
#from google.colab import drive
#drive.mount('/content/gdrive')
#sys.path.append('/content/gdrive/My Drive/Colab_assignment')
#!ls /content/gdrive/'My Drive'/Colab_assignment/*.py
#!cat '/content/gdrive/My Drive/Colab_assignment/calc_module.py' | head -5
# 認証処理
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
## 計算用
## ここは使い回すのであれば、クラス化するべき
def calc_alpha(mean):
  sigma = np.sqrt(round(mean * 0.1) ** 2) 
  return np.random.normal(mean,sigma)

def calc_damage(attack,defence, critial_prob, mode=None):
  critical = calc_critical(critial_prob)
  if mode is None:
    damage = (attack - defence) * critical
  elif mode == "fluction":
    mean = attack - defence
    # power then sqrt to change negative numbers to positive
    alpha = calc_alpha(mean) 
    damage = round((attack - defence + alpha) * critical)
    if damage <= 0: damage = 1 
  return damage

def calc_critical(critical_prob):
  return np.random.choice([1,2], p = [1 - critical_prob * 0.01, critical_prob * 0.01])

In [0]:
# dict_data = params_dataに変えたい
class ParamsCreater:
  def __init__(self):
    self.dict_data = {}

  def get_values(self,sheet, name,option=None):
    data =  sheet.get_all_values()
    headers = data.pop(0)
    df = pd.DataFrame(data, columns=headers)
    if option == 'numeric':
      df = df.apply(pd.to_numeric)
    df.set_index(df.columns[0],inplace=True)
    self.dict_data[name] = df

In [0]:
## まずはskillなしで組み込む
## logは全て配列に格納
class TurnBattleSim:
  ##Player, Enemy

  def __init__(self,data):
    self.set_params(data)

  def set_params(self,data,wave=1,stage=1, max_wave = 1):
    self.stage = stage
    self.wave = wave
    self.turn = 1
    self.MAX_WAVE = 0
    self.params = data
    ## A.ここもメソッド化しても良い
    self.player =  data['Player'].sort_values(by=['SPD'],ascending=False).copy()
    self.enemy  = data['Enemy'][data['Enemy'].Stage_Wave_ID == int(str(stage) + str(wave))].sort_values(by=['SPD'],ascending=False).copy()
    self.chara_master_original = self.player.append(self.enemy.drop(columns=['Stage','Stage_Wave_ID','Wave']))
    self.chara_master = self.chara_master_original.copy()
    self.chara_master[['original_ATK','original_DEF','original_SPD','original_CRT','original_HP']] = self.chara_master[['ATK','DEF','SPD','CRT','HP']]
    self.turn_index = self.chara_master.index

  ## skill generater的なのを作っても良い
  def set_skills(self,skill_master, chara_ids, skills):
    self.skill_master = skill_master
    self.skill_master.index =   self.skill_master.index.astype(int)
    self.buff = pd.DataFrame(columns=["character_id","skill_id","buff_turn","effect","target"],dtype=int)
    chara_ids = pd.Series(chara_ids)
    skills = pd.Series(skills)
    skill_df = pd.DataFrame(data={'character_id':chara_ids, 'skill_id':skills})

    for index, row in skill_df.iterrows():
      skill_df.at[index,"cool_time"] = self.skill_master.loc[row[1]]["cool_time"]
      skill_df["cool_time"] = skill_df["cool_time"]
      skill_df["current_cool_time"] = skill_df["cool_time"]
      skill_df["availability"] = True
    skill_df[['cool_time', 'current_cool_time']] = skill_df[['cool_time', 'current_cool_time']].astype(int)
    self.characters_skill = skill_df
  
  def reset_chara_params():
    self.character_master = self.chara_master_original.copy()

  def create_battle_log_df(self):
    ## 数が増える場合は拡張する必要あり
    base_columns = ['turn','chara_id', 'target_id', 'damage', 'used_skill']
    player_columns = [ 'player_' + str(i + 1) + '_HP' for i in range(len(self.player))]
    enemy_columns = [ 'enemy_' + str(i + 1) + '_HP'  for i in range(len(self.enemy))]
    skill_columns = [ 'skill_' + str(i + 1) for i in range(len(self.skill_master))]
    log_columns = base_columns + skill_columns + player_columns + enemy_columns
    self.battle_log = pd.DataFrame(columns=log_columns)

  def write_battle_log(self,character, target, damage,used_skill):
    base_list = [self.turn, character, target, damage,used_skill]
    skill_index = self.buff[self.buff["character_id"] == character].skill_id
    skills = [None] * len(sim.skill_master)
    if len(skill_index) == True:
      buff_skills = self.skill_master.loc[skill_index,'name'].values
      for i, skill in enumerate(buff_skills):
        skills[i] = skill
    players_hp = self.chara_master.query('side == 0')['HP'].to_list()
    enemies_hp = self.chara_master.query('side == 1')['HP'].to_list()
    log_list = base_list + skills + players_hp + enemies_hp
    log_series = pd.Series(log_list,index=self.battle_log.columns)
    self.battle_log = self.battle_log.append(log_series,ignore_index=True)
  ##battle logic methods
  def check_is_next_round(self,side, next_round):
    next_round = True
    if side == 0:
      next_round = False
    if self.wave + 1 >  self.MAX_WAVE:
      next_round = False  
    return next_round

  def generate_next_wave(self,stage,wave):
      next_wave_enemy = self.enemy[self.enemy.Stage_Wave_ID == int(str(stage) + str(wave))].copy()
      self.chara_master_original = self.chara_master.append(next_wave_enemy.drop(columns=['Stage','Stage_Wave_ID','Wave']))
      self.chara_master = self.chara_master_original.copy()
      self.chara_master[['original_ATK','original_DEF','original_SPD','original_CRT','original_HP']] = self.chara_master[['ATK','DEF','SPD','CRT','HP']]

  ## skill_things
  def use_skill(self, skill_index,player_side, enemy_side):
    skill = self.skill_master.loc[skill_index]
    skill_dict = {}
    if skill.Type == 'heal':
      targets = self.chara_master.query("side == @player_side and ST== 1 and HP != original_HP").sort_values(by=['HP'],ascending=True)
      if len(targets) == False: return 
      target_index = targets.index[0]
      max_HP = self.chara_master.loc[target_index, 'original_' + skill.target]
      HP = self.chara_master.loc[target_index, skill.target] + 200
      self.chara_master.loc[target_index, skill.target] =  max_HP if  HP >= max_HP else HP
      self.target = target_index
    else:
      side = player_side if skill.Type == 'buff' else enemy_side
      target_index = self.chara_master.query("side == @side and ST== 1").index
      for i in target_index:
        if len(self.buff.query("character_id == @i and skill_id == @skill_index")) == True: next #既にかかってる場合はskip
        skill_dict["character_id"] = i
        skill_dict["skill_id"] = skill_index
        skill_dict["buff_turn"] = int(skill.buff_turn)
        skill_dict["effect"] = float(skill.effect)
        skill_dict["target"] = skill.target 
        self.buff = self.buff.append(skill_dict,ignore_index=True)
      param = self.chara_master.loc[target_index, skill.target]
      self.chara_master.loc[target_index, skill.target] = param * float(skill.effect)
    return skill["name"]

  def check_buff(self,id):
    target_index = self.buff.query("character_id == @id").index
    if len(target_index) == False: return None
    for i in target_index: ## loopはいずれ改善
      self.buff.loc[i, "buff_turn"] = self.buff.loc[i, "buff_turn"] - 1
      if self.buff.loc[i, "buff_turn"] < 0:
        target_param = self.buff.loc[i, "target"]
        param_value = self.chara_master.loc[id, target_param] / self.buff.loc[i, "effect"]
        self.chara_master.loc[id, self.buff.loc[i, "target"]] = param_value
        self.buff.drop(i,axis=0,inplace=True)

  def check_cool_time(self, chara_id):
    unavailable_skills = self.characters_skill.query("character_id == @chara_id and availability == False")["skill_id"].values
    if len(unavailable_skills) == True:
      for i in unavailable_skills:
        ## Arrayにまとめて最後に更新の方がわかりやすい
        target_index = self.characters_skill.query('skill_id ==  @i').index
        current_cool_time =  self.characters_skill.loc[target_index, "current_cool_time"] - 1
        self.characters_skill.loc[target_index, "current_cool_time"] = current_cool_time
        if self.characters_skill.loc[target_index, "current_cool_time"].values[0] == 0:
          current_cool_time = self.characters_skill.loc[target_index, "cool_time"]
          self.characters_skill.loc[target_index, "current_cool_time"] = current_cool_time
          self.characters_skill.loc[target_index, "availability"] = True

  ## battle logic
  def battle(self):
    next_round = True
    turn_reset = True
    chara_turn_index = self.chara_master.query("ST==1 and turn == 1").sort_values(by=['SPD'],ascending=False).index
    for id in chara_turn_index:
      ## ここもrefactor
      character = self.chara_master.loc[id]
      side = character.side
      target = np.random.choice(self.chara_master.query("side != @side and ST== 1").index)
      self.target = target
      enemy = self.chara_master.loc[target]
      used_skill = None
      ## skill check
      available_skills = self.characters_skill.query("character_id == @id and availability == True")
      if character.has_skill == 1 and len(available_skills) == True:
        skills = available_skills["skill_id"]
        if len(skills) == True:
          skill_index = np.random.choice(skills.values)
          self.check_cool_time(id)
          self.check_buff(id)
          used_skill = self.use_skill(skill_index, side, enemy.side)
          self.characters_skill.loc[self.characters_skill["skill_id"] == skill_index, "availability"] = False
        damage = 0
        HP = enemy.HP
      else:
        if character.has_skill == 1: self.check_cool_time(id)
        damage = calc_damage(character.ATK, enemy.DEF, character.CRT,mode="fluction")
        HP = enemy.HP - damage
        self.check_buff(id)
  
      self.chara_master.at[id,"turn"]  = 0
      ## ここrefactor
      if HP <= 0:
        self.chara_master.loc[target,"HP"] = 0
        self.chara_master.loc[target, 'ST'] = 0
        self.write_battle_log(id, self.target, damage,used_skill)
        if self.chara_master.query("side != @side").ST.sum() == 0: 
          next_round = self.check_is_next_round(side, next_round)
          if next_round is False: 
            break #next turn False 
          self.wave = self.wave + 1
          self.generate_next_wave(stage,wave)
          turn_reset = False
          break
      else:
        self.chara_master.at[target, "HP"] = HP 
        self.write_battle_log(id, self.target, damage, used_skill)
    if turn_reset == True: self.chara_master["turn"] = 1 #Turnの回復
    if next_round is True: 
      self.turn = self.turn + 1
      return self.battle()
    return int(character.side)  

  def start_battle(self):
    self.create_battle_log_df()
    side = self.battle()
    # required to modify log system
    self.battle_log = self.battle_log.fillna(value=pd.np.nan)
    skill_columns = self.battle_log.columns[pd.Series(self.battle_log.columns.str.startswith('skill'))]
    self.battle_log[skill_columns] = self.battle_log[skill_columns].astype(object)
    float_columns = self.battle_log.select_dtypes(float).columns
    self.battle_log[float_columns] = self.battle_log[float_columns].astype(int)
    return side

In [0]:
### ここからクラスを呼び出して使用

In [0]:
## Parameterの作成
creater = ParamsCreater()
creater.get_values(gc.open('battle_sim_refactor').worksheet('Player'),'Player',option='numeric')
creater.get_values(gc.open('battle_sim_refactor').worksheet('Enemy'),'Enemy',option='numeric')
creater.get_values(gc.open('battle_sim_refactor').worksheet('skill_master'),'skill_master')

In [0]:
## 設定の確認
print('味方———————————')
print(creater.dict_data['Player'])
print('\n敵———————————')
print(creater.dict_data['Enemy'])

味方———————————
          ATK  DEF  SPD  CRT   HP  ST  has_skill  side  turn
Chara_id                                                    
1          80   10    9    9  200   1          0     0     1
2          10   80    1    1  600   1          1     0     1
3          40   40   10   10  350   1          0     0     1
4          65   10   20    5  300   1          1     0     1
5          60    5    5   30  450   1          1     0     1

敵———————————
          ATK  DEF  SPD  CRT    HP  ...  Stage_Wave_ID  ST  has_skill  side  turn
Chara_id                            ...                                          
111        20   30    2   10   500  ...             11   1          0     1     1
112        35   30   10   20   750  ...             12   1          0     1     1
121        25   15    7   20   700  ...             21   1          0     1     1
122        25   15    7   20   700  ...             22   1          0     1     1
222       100   20    3   15   500  ...             2

In [0]:
## TurnBattleSimクラスのインスタンス化
sim = TurnBattleSim(creater.dict_data)
sim.set_params(creater.dict_data,wave=0,stage=1, max_wave=0) #このインスタンスで使用するPlayerとEnemyを設定

chara_ids = [2,4,5]
skills = [1,2,3]
sim.set_skills(creater.dict_data["skill_master"],chara_ids,skills) #このインスタンスで使用するPlayerとEnemyを設定

In [0]:
# シミュレーションの設定確認
print(sim.player)
print(sim.enemy)

          ATK  DEF  SPD  CRT   HP  ST  has_skill  side  turn
Chara_id                                                    
4          65   10   20    5  300   1          1     0     1
3          40   40   10   10  350   1          0     0     1
1          80   10    9    9  200   1          0     0     1
5          60    5    5   30  450   1          1     0     1
2          10   80    1    1  600   1          1     0     1
          ATK  DEF  SPD  CRT    HP  ...  Stage_Wave_ID  ST  has_skill  side  turn
Chara_id                            ...                                          
101        40   10    6   10  3000  ...             10   1          0     1     1
103        40   20    3   15   500  ...             10   1          0     1     1
102        20   30    2   10   350  ...             10   1          0     1     1

[3 rows x 12 columns]


In [0]:
# シミュレーションの実行
win_side = sim.start_battle()
print('You won!') if win_side == 0 else print('Enemy won')

You won!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:187: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


In [0]:
# ログ確認
sim.battle_log

,turn,chara_id,target_id,damage,used_skill,skill_1,skill_2,skill_3,player_1_HP,player_2_HP,player_3_HP,player_4_HP,player_5_HP,enemy_1_HP,enemy_2_HP,enemy_3_HP
0,1,4,101,0,DEF_DOWN,NaN,NaN,NaN,300,350,200,450,600,3000,500,350
1,1,3,101,72,NaN,NaN,NaN,NaN,300,350,200,450,600,2928,500,350
2,1,1,101,147,NaN,NaN,NaN,NaN,300,350,200,450,600,2781,500,350
3,1,101,1,58,NaN,DEF_DOWN,NaN,NaN,300,350,142,450,600,2781,500,350
4,1,5,1,0,HEAL,NaN,NaN,NaN,300,350,200,450,600,2781,500,350
5,1,103,1,61,NaN,DEF_DOWN,NaN,NaN,300,350,139,450,600,2781,500,350
6,1,102,1,19,NaN,DEF_DOWN,NaN,NaN,300,350,120,450,600,2781,500,350
7,1,2,103,0,ATK_UP,ATK_UP,NaN,NaN,300,350,120,450,600,2781,500,350
8,2,4,103,141,NaN,ATK_UP,NaN,NaN,300,350,120,450,600,2781,359,350
9,2,3,103,80,NaN,ATK_UP,NaN,NaN,300,350,120,450,600,2781,279,350


In [0]:
## キャラクターの設定を変えて実行してみる
##  同じくクラスを呼び出してインスタンス化
creater.dict_data['Player'].loc[3,'ATK'] = 10000

sim_2 = TurnBattleSim(creater.dict_data)
sim_2.set_params(creater.dict_data,wave=0,stage=1, max_wave=0) #このインスタンスで使用するPlayerとEnemyを設定

chara_ids = [2,4,5]
skills = [1,2,3]
sim_2.set_skills(creater.dict_data["skill_master"],chara_ids,skills) #このインスタンスで使用するPlayerとEnemyを設定

In [0]:
# シミュレーションの設定確認
# Chara_id 3のATKが10,000になっている
print(sim_2.player)
print(sim_2.enemy)

            ATK  DEF  SPD  CRT   HP  ST  has_skill  side  turn
Chara_id                                                      
4            65   10   20    5  300   1          1     0     1
3         10000   40   10   10  350   1          0     0     1
1            80   10    9    9  200   1          0     0     1
5            60    5    5   30  450   1          1     0     1
2            10   80    1    1  600   1          1     0     1
          ATK  DEF  SPD  CRT    HP  ...  Stage_Wave_ID  ST  has_skill  side  turn
Chara_id                            ...                                          
101        40   10    6   10  3000  ...             10   1          0     1     1
103        40   20    3   15   500  ...             10   1          0     1     1
102        20   30    2   10   350  ...             10   1          0     1     1

[3 rows x 12 columns]


In [0]:
# シミュレーション2の実行
win_side = sim_2.start_battle()
print('You won!') if win_side == 0 else print('Enemy won')

You won!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:187: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


In [0]:
## Logの確認
sim_2.battle_log

,turn,chara_id,target_id,damage,used_skill,skill_1,skill_2,skill_3,player_1_HP,player_2_HP,player_3_HP,player_4_HP,player_5_HP,enemy_1_HP,enemy_2_HP,enemy_3_HP
0,1,4,103,0,DEF_DOWN,NaN,NaN,NaN,300,350,200,450,600,3000,500,350
1,1,3,103,20671,NaN,NaN,NaN,NaN,300,350,200,450,600,3000,0,350
2,1,1,101,149,NaN,NaN,NaN,NaN,300,350,200,450,600,2851,0,350
3,1,101,4,62,NaN,DEF_DOWN,NaN,NaN,238,350,200,450,600,2851,0,350
4,1,5,4,0,HEAL,NaN,NaN,NaN,300,350,200,450,600,2851,0,350
5,1,103,2,1,NaN,DEF_DOWN,NaN,NaN,300,350,200,450,599,2851,0,350
6,1,102,2,1,NaN,DEF_DOWN,NaN,NaN,300,350,200,450,598,2851,0,350
7,1,2,102,0,ATK_UP,ATK_UP,NaN,NaN,300,350,200,450,598,2851,0,350
8,2,4,102,143,NaN,ATK_UP,NaN,NaN,300,350,200,450,598,2851,0,207
9,2,3,101,26251,NaN,ATK_UP,NaN,NaN,300,350,200,450,598,0,0,207
